# word clouds are the wedding font of visualization

how do we make word clouds that aren't toys?

In [1]:
%%

    shell.environment.globals.update(vars(__builtins__))

In [2]:
%%
use the `wordcloud` package to generate a table of word cloud data to visual

    import wordcloud
    cloud = wordcloud.WordCloud()

using the script from george carlin's 1972 <q>the seven words you can never say on television</q>
https://genius.com/George-carlin-the-seven-words-you-can-never-say-on-television-annotated

    cloud.generate_from_text(Path("seven_words.txt").read_text())

In [3]:
%%
create a structured dataframe from the text

{{df.T.style.to_html()}}

    df = DataFrame(cloud.layout_)
    df = df[0].apply(list).series().rename(columns=dict(zip(df.columns, "word ratio".split()))).combine_first(
        df[df.columns[1:]]
    ).set_index("word").rename(columns={1:"ct", 2: "position", 3: "rotate", 4: "color"})
    df.rotate = df.rotate.replace({2: 90, numpy.nan: 0})
    df = df.combine_first(df.pop("position").apply(list).series().rename(columns={0:"x", 1: "y"}))

<details>
<summary>
handling image rotations in the data
</summary>
 
{{df.rotate.dropna().value_counts().to_frame("rotations").style.set_caption(
    "frequency of rotations in the parent dataframe"
).to_html()}}

    >>> assert __import__("PIL").Image.ROTATE_90 == 2 

</details>

In [4]:
%%
create the word cloud

<style>
#cloud {
    --width: 1000px;
    --height: 600px;
    position: relative;
    top: 0;
    left: 0;
    display: block;
    height: var(--height);
    width: var(--width);
    thead {
        display: none; /** should be visuall hidden **/
    }
    tbody {
        transform-style: preserve-3d;
        display: block;
        position: relative;
        top: 0;
        left: 0;
        transform-origin: top left;
        tr {
            top: 0;
            left: 0;
            th, td {
                background: unset;
                display: block;
                position: absolute;
                font-size: calc((var(--r) * (1 - .1) + .1) * 10rem);
                span:not([aria-hidden]) {
                    position: absolute;
                    display: inline-block;
                    transform: 
                        rotate3d(1,1,1,15deg)
                        translateX(calc(var(--x)/var(--x-max)*var(--width)))
                        translateY(calc(var(--y)/var(--y-max)*var(--height)))
                        translateZ(calc(var(--z, 1) * 500px));
                    opacity: var(--z);
                    color: var(--color);
                }
            }
            td {
                display: none;
            }
            th:is(:hover, :focus, :focus-within) ~ td.ct {
                filter: invert(100%);
                display: inline-block;
            }
            th span:not([aria-hidden])::after {
                content: var(--word);
            }
            th > * ~ span:last-of-type::after {
                -webkit-text-stroke: 2px black;
                text-stroke: 2px black;
            }
        }
    }
}
</style>
<table id=cloud  style="--x-min: {{df.x.min()}}; --x-max: {{df.x.max()}};
--y-min: {{df.y.min()}}; --y-max: {{df.y.max()}};">
    <thead>
        <tr><th scope="col">word</th>{% for x in df.columns %}<th scope="col">{{x}}</th>{% endfor %}</tr>
     </thead>
    <tbody>{% for i, row in df.head(100)[::-1].iterrows() %}
        <tr style="--x: {{row.x}}; --y: {{row.y}}; --rot: {{row.rotate}}; --r: {{row.ratio}}; --word: '{{i.lower()}}'; --color: {{row.color}};">
            <th scope="row">
            <span aria-hidden=true>
            {%- for inc in (range(2,row.ct)) -%}
            <span style="--z: {{inc/row.ct}}"></span>
            {%- endfor -%}</span>
            <span style="--z: 1;"></span>
            </th>
            {%- for k, value in row.items() -%}
            <td class="{{k}}"><br/><span>{{value}}</span></td>
        {%- endfor -%}</tr>{% endfor -%}
    </tbody>
</table>

word,color,ct,ratio,rotate,x,y
,"rgb(72, 22, 104)",5,0.030303030303030304,0.0,92,164
,"rgb(58, 84, 140)",5,0.030303030303030304,0.0,78,237
,"rgb(56, 89, 140)",5,0.030303030303030304,0.0,7,382
,"rgb(32, 146, 140)",5,0.030303030303030304,0.0,21,383
,"rgb(96, 202, 96)",5,0.030303030303030304,0.0,22,226
,"rgb(144, 215, 67)",5,0.030303030303030304,0.0,130,113
,"rgb(132, 212, 75)",5,0.030303030303030304,0.0,145,96
,"rgb(51, 99, 141)",5,0.030303030303030304,0.0,23,81
,"rgb(157, 217, 59)",5,0.030303030303030304,90.0,135,391
,"rgb(59, 82, 139)",5,0.030303030303030304,0.0,2,61
